In [ ]:
import sys
import AE_models

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import numpy as np
import keras 
from keras import layers
import scipy.io as sio
import pandas as pd 

import matplotlib.pyplot as plt

In [ ]:
def value_range_analysis(X):
    if sum(sum(((np.round(X,4) <= 1)) & (X >= 0))) == X.shape[0] * X.shape[1]:
        print("Range value: [0,1]")
    else:
        print("ERROR!!")

In [ ]:
def plotLossesLayers(h_layers, dicc, carpeta):
    plot_layers = []
    for i in range(len(h_layers)):
        plot_layers.append(h_layers[i][1])

    keys = list(dicc.keys())
    
    filas = int(np.ceil(len(keys)/2))
    columnas = int(np.sqrt(np.round(len(keys)/2)))
    
    f, axis = plt.subplots(filas, columnas, figsize=(20,20))
    index_keys = 0
    for i in range(filas):
        for j in range(columnas):
            axis[i,j].plot(plot_layers, dicc[keys[index_keys]][0])
            axis[i,j].plot(plot_layers, dicc[keys[index_keys]][1])
            axis[i,j].legend(["val_loss", "loss"])
            axis[i,j].set_title(keys[i])
            index_keys += 1

    plt.close(f)

## Final model

In [ ]:
folders = ["S1", "S2", "S3", "S4", "S5"]
normalizar = True
plotAndSaveImg = False

for index_data in range(len(folders)):
    print("================================================================>" + folders[index_data] + "<=======================================================")
    X_train_original, X_test_original, y_train_original, y_test_original = AE_models.loadData(index_data, normalizar, folders)
    value_range_analysis(X_train_original)

    autoencodertype = {'DAE': True, 'AE': False}


    epochs = 5000
    initial_learning_rate = [1e-4, 1e-3, 1e-2, 1e-1]
    batch_size = [32]
    dropout=[0, .05, .1]
    std_noise = [0.01, 0.025, 0.05]
    h_layers = [[X_train_original.shape[0], 525],
                [X_train_original.shape[0], 500],
                [X_train_original.shape[0], 475],
                [X_train_original.shape[0], 450],
                [X_train_original.shape[0], 425],
                [X_train_original.shape[0], 400],
                [X_train_original.shape[0], 375],
                [X_train_original.shape[0],  350],
                [X_train_original.shape[0],  325],
                [X_train_original.shape[0],  300],
                [X_train_original.shape[0], 275]]


    hyperparameters = {'epochs': epochs,
                       'initial_learning_rate':initial_learning_rate,
                       'kfold':5,
                       "mindelta": 0.0001}

    y_pre_train = y_train_original
    y_test = y_test_original
    X_pre_train = X_train_original
    X_test = X_test_original

    semillas = [9,18,35, 52, 75]

    kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=semillas[index_data])
    kf.get_n_splits(X_pre_train)

    bestHyperparameters = {'h_layers': -1, 'batch_size': -1, 'initial_learning_rate': -1 }

    bestMetricDev = np.inf
    dicc = {}
    for bs in range(len(batch_size)):
        for d in range(len(dropout)):
            for init in range(len(initial_learning_rate)):
                for std in range(len(std_noise)):
                    plot_val_loss = []
                    plot_loss = []
                    for l in range(len(h_layers)):
                        hyperparameters_aux = {'epochs': hyperparameters['epochs'],
                                           'initial_learning_rate': hyperparameters['initial_learning_rate'][init],
                                           "mindelta": hyperparameters["mindelta"],
                                           'batch_size': batch_size[bs],
                                           'h_layers': h_layers[l],
                                           'dropout': dropout[d],
                                           'std_noise': std_noise[std],
                                           'verbose': 0}

                        v_early = []
                        v_metric_dev = []
                        v_hist = []
                        v_val_loss = []
                        for train_index, val_index in kf.split(X_pre_train):
                            X_train, X_val = X_pre_train[train_index], X_pre_train[val_index]
                            AE_models.reset_keras()
                            model, hist, early = AE_models.runNetwork(X_train, X_val,
                                                                      hyperparameters_aux,
                                                                      autoencodertype)


                            v_early.append(early)
                            v_hist.append(hist)
                            v_val_loss.append(np.min(hist.history["val_loss"]))
                        metric_dev = np.mean(v_val_loss)
                        plot_val_loss.append(metric_dev)
                        plot_loss.append(np.mean(hist.history["loss"]))

                        if metric_dev < bestMetricDev:
                            print("\tChange the best ", bestMetricDev, " by metric dev: ", metric_dev)
                            bestMetricDev = metric_dev
                            bestHyperparameters['h_layers'] = l
                            bestHyperparameters['batch_size'] = bs
                            bestHyperparameters['initial_learning_rate'] = init
                            bestHyperparameters['std_noise'] = std
                            bestHyperparameters['dropout'] = d
                            print("%%%%%%%%%bestHyperparameters%%%%%%%%%")
                            print(bestHyperparameters)

                    dicc["bs:" + str(batch_size[bs]) + "-lr:" + str(initial_learning_rate[init]) + "-std:" + str(std_noise[std])] = [plot_val_loss, plot_loss]



    if plotAndSaveImg:
        plotLossesLayers(h_layers, dicc, folders[index_data])

    print("Layers selected:", h_layers[bestHyperparameters["h_layers"]])
    print("batch_size selected:", batch_size[bestHyperparameters["batch_size"]])
    print("initial_learning_rate selected:", initial_learning_rate[bestHyperparameters["initial_learning_rate"]])
    print("std_noise selected:", std_noise[bestHyperparameters["std_noise"]])
    print("std_noise selected:", dropout[bestHyperparameters["dropout"]])

    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%================================================================> BEST MODEL SELECT WITH DIFFERENTS HYPERPARAMETERS <=======================================================%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    hyperparameters = {'epochs': hyperparameters['epochs'],
                   'initial_learning_rate': initial_learning_rate[bestHyperparameters["initial_learning_rate"]],
                   "mindelta": hyperparameters["mindelta"],
                   'batch_size': batch_size[bestHyperparameters["batch_size"]],
                   'h_layers': h_layers[bestHyperparameters["h_layers"]],
                   'std_noise': std_noise[bestHyperparameters["std_noise"]],
                   'dropout': std_noise[bestHyperparameters["dropout"]],
                   'verbose': 1}
    
    AE_models.reset_keras()
    X_train_aux, X_val_aux = train_test_split(X_pre_train,
                                              test_size=0.3,
                                              random_state=semillas[index_data])
    
    autoencoder, hist, early = AE_models.runNetwork(X_train_aux, X_val_aux,
                                                    hyperparameters,
                                                    autoencodertype)



    encoder = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)

    X_train = sio.loadmat('../../data_reduced_by_tck/' + folders[index_data] + '/Ktrtr')
    X_test = sio.loadmat('../../data_reduced_by_tck/' + folders[index_data] + '/Ktrte')
    X_train = X_train['Ktrtr']
    X_test = X_test['Ktrte']
    X_test = X_test.T

    X_train, X_test = AE_models.normData_minmax(X_train, X_test)

    encoder = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    X_train_encode = encoder.predict(X_train)
    X_test_encode = encoder.predict(X_test)

    pd.DataFrame(X_train_encode).to_csv("../../1_Clasifications_models/data_reduced/DAE/X_train_Norm_DAE_" + folders[index_data] + ".csv", index=False)
    pd.DataFrame(X_test_encode).to_csv("../../1_Clasifications_models/data_reduced/DAE/X_test_Norm_DAE_" + folders[index_data] + ".csv", index=False)